In [1]:
!pip install flask flask-cors requests termcolor pyngrok -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## MCP Server & DB Setup 

**DB Setup**

In [2]:
# DB connection setup
import sqlite3
import json
from datetime import datetime
from typing import Optional, Dict, List, Any

DB_PATH = 'support.db'

def get_db_connection():
    """Create a database connection with row factory for dict-like access."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row  # This allows us to access columns by name
    return conn

def row_to_dict(row: sqlite3.Row) -> Dict[str, Any]:
    """Convert a SQLite row to a dictionary."""
    return {key: row[key] for key in row.keys()}

In [3]:
# Tools for database operations

# get_customer: Retrieve a specific customer by ID
def get_customer(customer_id: int) -> Dict[str, Any]:
    """
    Retrieve a specific customer by ID.
    
    Args:
        customer_id: The unique ID of the customer
        
    Returns:
        Dict containing customer data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        
        if row:
            return {
                'success': True,
                'customer': row_to_dict(row)
            }
        else:
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# list_customers: List all customers, optionally filtered by status
def list_customers(status: Optional[str] = None) -> Dict[str, Any]:
    """
    List all customers, optionally filtered by status.
    
    Args:
        status: Optional filter - 'active', 'disabled', or None for all
        
    Returns:
        Dict containing list of customers or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        if status:
            if status not in ['active', 'disabled']:
                return {
                    'success': False,
                    'error': 'Status must be "active" or "disabled"'
                }
            cursor.execute('SELECT * FROM customers WHERE status = ? ORDER BY name', (status,))
        else:
            cursor.execute('SELECT * FROM customers ORDER BY name')
        
        rows = cursor.fetchall()
        conn.close()
        
        customers = [row_to_dict(row) for row in rows]
        
        return {
            'success': True,
            'count': len(customers),
            'customers': customers
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# update_customer: Update customer information
def update_customer(customer_id: int, name: Optional[str] = None, 
                   email: Optional[str] = None, phone: Optional[str] = None) -> Dict[str, Any]:
    """
    Update customer information.
    
    Args:
        customer_id: The unique ID of the customer to update
        name: New name (optional)
        email: New email (optional)
        phone: New phone (optional)
        
    Returns:
        Dict containing updated customer data or error message
    """
    try:
        # Check if customer exists
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
        
        # Build update query dynamically based on provided fields
        updates = []
        params = []

        if name is not None:
            updates.append('name = ?')
            params.append(name.strip())
        if email is not None:
            updates.append('email = ?')
            params.append(email)
        if phone is not None:
            updates.append('phone = ?')
            params.append(phone)
        
        if not updates:
            conn.close()
            return {
                'success': False,
                'error': 'No fields to update'
            }
        
        # Always update the updated_at timestamp
        updates.append('updated_at = CURRENT_TIMESTAMP')
        params.append(customer_id)
        
        update_clause = ', '.join(updates)
        query = f'UPDATE customers SET {update_clause} WHERE id = ?'
        cursor.execute(query, params)
        conn.commit()
        
        # Fetch updated customer
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        
        return {
            'success': True,
            'message': f'Customer {customer_id} updated successfully',
            'customer': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# create_ticket: Create a new support ticket for a customer
def create_ticket(customer_id, issue, priority) -> Dict[str, Any]:
    """
    Create a new support ticket for a customer.
    
    Args:
        customer_id: The unique ID of the customer
        issue: Description of the issue
        priority: Priority level ('low', 'medium', 'high')
    Returns:
        Dict containing ticket data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Check if customer exists
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
        
        # Insert new ticket
        cursor.execute(
            'INSERT INTO tickets (customer_id, issue, priority, status, created_at, updated_at) '
            'VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)',
            (customer_id, issue, priority, 'open')
        )
        conn.commit()
        
        ticket_id = cursor.lastrowid
        
        # Fetch the newly created ticket
        cursor.execute('SELECT * FROM tickets WHERE id = ?', (ticket_id,))
        row = cursor.fetchone()
        conn.close()
        
        return {
            'success': True,
            'message': f'Ticket {ticket_id} created successfully',
            'ticket': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# get_customer_history: Retrieve support ticket history for a customer
def get_customer_history(customer_id):
    """
    Retrieve the support ticket history for a specific customer.
    
    Args:
        customer_id: The unique ID of the customer
    Returns:
        Dict containing list of tickets or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Check if customer exists
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
        
        # Fetch tickets for the customer
        cursor.execute('SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC', (customer_id,))
        rows = cursor.fetchall()
        conn.close()
        
        tickets = [row_to_dict(row) for row in rows]
        
        return {
            'success': True,
            'count': len(tickets),
            'tickets': tickets
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

In [4]:
# Quick test - Fetching customer ID 1
print("\nQuick test - Fetching customer ID 1:")
result = get_customer(1)
if result['success']:
    customer = result['customer']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")


Quick test - Fetching customer ID 1:
   Name: John Doe
   Email: john.doe@example.com
   Status: active


**MCP HTTP Streaming Server Implementation**

In [5]:
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
import json
import threading
import time
from typing import Dict, Any, Generator

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Server state
server_thread = None
server_running = False

# MCP Protocol Implementation

# Define the tools that will be exposed via MCP
MCP_TOOLS = [
    {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "list_customers",
        "description": "List all customers in the database. Can optionally filter by status (active or disabled).",
        "inputSchema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "Optional filter by customer status"
                }
            }
        }
    },
    {
        "name": "update_customer",
        "description": "Update an existing customer's information. Provide the customer ID and the fields to update.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to update"
                },
                "name": {
                    "type": "string",
                    "description": "New name (optional)"
                },
                "email": {
                    "type": "string",
                    "description": "New email (optional)"
                },
                "phone": {
                    "type": "string",
                    "description": "New phone (optional)"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "create_ticket",
        "description": "Create a new support ticket for a customer. Requires customer ID, issue description, and priority level.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer"
                },
                "issue": {
                    "type": "string",
                    "description": "Description of the issue"
                },
                "priority": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Priority level of the ticket"
                }
            },
            "required": ["customer_id", "issue", "priority"]
        }
    },
   {
       "name": "get_customer_history",
       "description": "Retrieve the support ticket history for a specific customer by their ID.",
         "inputSchema": {
              "type": "object",
              "properties": {
                "customer_id": {
                     "type": "integer",
                     "description": "The unique ID of the customer whose history to retrieve"
                }
              },
              "required": ["customer_id"]
        }
   }
]

def create_sse_message(data: Dict[str, Any]) -> str:
    """
    Format a message for Server-Sent Events (SSE).
    SSE format: 'data: {json}\n\n'
    """
    return f"data: {json.dumps(data)}\n\n"

def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle MCP initialize request.
    This is the first message in the MCP protocol handshake.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "tools": {},  # We support tools
            },
            "serverInfo": {
                "name": "customer-management-server",
                "version": "1.0.0"
            }
        }
    }

def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/list request.
    Returns the list of available tools.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "tools": MCP_TOOLS
        }
    }

def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/call request.
    Executes the requested tool and returns the result.
    """
    params = message.get("params", {})
    tool_name = params.get("name")
    arguments = params.get("arguments", {})
    
    # Map tool names to functions
    tool_functions = {
        "get_customer": get_customer,
        "list_customers": list_customers,
        "update_customer": update_customer,
        "create_ticket": create_ticket,
        "get_customer_history": get_customer_history,
    }
    
    if tool_name not in tool_functions:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Tool not found: {tool_name}"
            }
        }
    
    try:
        # Call the tool function with the provided arguments
        result = tool_functions[tool_name](**arguments)
        
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": json.dumps(result, indent=2)
                    }
                ]
            }
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32603,
                "message": f"Tool execution error: {str(e)}"
            }
        }

def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process an MCP message and route it to the appropriate handler.
    """
    method = message.get("method")
    
    if method == "initialize":
        return handle_initialize(message)
    elif method == "tools/list":
        return handle_tools_list(message)
    elif method == "tools/call":
        return handle_tools_call(message)
    else:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Method not found: {method}"
            }
        }

# Flask Routes

@app.route('/mcp', methods=['POST'])
def mcp_endpoint():
    """
    Main MCP endpoint for MCP communication.
    Receives MCP messages and streams responses using Server-Sent Events.
    """
    # Get the MCP message from the request BEFORE entering the generator
    # This must be done in the request context
    message = request.get_json()
    
    def generate():
        try:
            print(f"📥 Received MCP message: {message.get('method')}")
            
            # Process the message
            response = process_mcp_message(message)
            
            print(f"📤 Sending MCP response")
            
            # Send the response as SSE
            yield create_sse_message(response)
            
        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": None,
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)
    
    return Response(generate(), mimetype='text/event-stream')

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint to verify server is running."""
    return jsonify({
        "status": "healthy",
        "server": "customer-management-mcp-server",
        "version": "1.0.0"
    })

print("MCP Server implementation complete!")
print("\n🔧 Server features:")
print("   - MCP protocol support (2024-11-05)")
print("   - Server-Sent Events (SSE) streaming")
print(f"   - {len(MCP_TOOLS)} tools exposed")
print("   - Health check endpoint")
print("   - CORS enabled for cross-origin requests")

MCP Server implementation complete!

🔧 Server features:
   - MCP protocol support (2024-11-05)
   - Server-Sent Events (SSE) streaming
   - 5 tools exposed
   - Health check endpoint
   - CORS enabled for cross-origin requests


**Start the MCP Server**

In [ ]:
import threading
import time
import requests
from termcolor import colored
import os
from pyngrok import ngrok

# Server configuration
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'

NGROK_AUTHTOKEN = "365ImSzzzTP9nb4WAOnZDiaNPNp_9UVqqneqDmvm5xUZymcW"

def run_server():
    """Run the Flask server in a separate thread."""
    global server_running
    server_running = True
    app.run(host=SERVER_HOST, port=SERVER_PORT, debug=False, use_reloader=False)

def start_server(use_ngrok=True):
    """Start the MCP server in a background thread."""
    global server_thread, server_running
    
    if server_thread and server_thread.is_alive():
        print(colored("⚠️  Server is already running!", "yellow"))
        return
    
    print(colored("🚀 Starting MCP server...", "cyan"))
    
    # Start server in background thread
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    
    # Wait for server to start
    time.sleep(2)
    
    # Check if server is healthy
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=5)
        if response.status_code == 200:
            print(colored("✅ MCP Server is running!", "green"))
            print(colored(f"📍 Local URL: {SERVER_URL}", "cyan"))
            
            # Set up ngrok tunnel if requested
            if use_ngrok:
                print(colored("\n🌐 Setting up public tunnel with ngrok...", "cyan"))
                try:
                    try:
                        ngrok.set_auth_token(NGROK_AUTHTOKEN) 
                        print(colored("✅ Ngrok authenticated", "green"))
                    except Exception as e:
                        print(colored("⚠️  NGROK_AUTHTOKEN not found in Colab secrets", "yellow"))
                        print(colored("   To use ngrok:", "yellow"))
                        print(colored("   1. Get free authtoken from https://ngrok.com", "yellow"))
                        print(colored("   2. In Colab: Click 🔑 (Secrets) in left sidebar", "yellow"))
                        print(colored("   3. Add secret: Name='NGROK_AUTHTOKEN', Value=<your-token>", "yellow"))
                        print(colored("   4. Enable 'Notebook access' for the secret", "yellow"))
                        print(colored("   5. Re-run this cell", "yellow"))
                        print(colored("\n   Server is still accessible locally at " + SERVER_URL, "cyan"))
                        return
                    
                    # Create ngrok tunnel
                    public_url = ngrok.connect(SERVER_PORT)
                    print(colored(f"✅ Public URL: {public_url}", "green", attrs=["bold"]))
                    print(colored(f"📍 MCP Endpoint: {public_url}/mcp", "green", attrs=["bold"]))
                    print(colored(f"📍 Health Check: {public_url}/health", "cyan"))
                    print()
                    print(colored("🔍 MCP Inspector Instructions:", "yellow", attrs=["bold"]))
                    print(colored("1. Run in terminal: npx @modelcontextprotocol/inspector", "yellow"))
                    print(colored("2. This will open MCP Inspector in your browser", "yellow"))
                    print(colored(f"3. Enter MCP URL: {public_url}/mcp", "yellow"))
                    print(colored("4. Click 'Connect' and test the customer management tools!", "yellow"))
                except Exception as e:
                    if "NGROK_AUTHTOKEN" not in str(e):
                        print(colored(f"⚠️  Could not set up ngrok tunnel: {e}", "yellow"))
                        print(colored("   Server is still accessible locally", "yellow"))
        else:
            print(colored("❌ Server started but health check failed", "red"))
    except Exception as e:
        print(colored(f"❌ Failed to connect to server: {e}", "red"))

def stop_server():
    """Stop the MCP server."""
    global server_running
    server_running = False
    print(colored("🛑 Server stopped", "yellow"))
    print(colored("   Note: In Colab, the thread will continue until the runtime is reset", "yellow"))

def check_server_status():
    """Check if the server is running."""
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=2)
        if response.status_code == 200:
            print(colored("✅ Server is running and healthy", "green"))
            health_data = response.json()
            print(f"   Status: {health_data['status']}")
            print(f"   Server: {health_data['server']}")
            print(f"   Version: {health_data['version']}")
            return True
        else:
            print(colored("❌ Server is not responding correctly", "red"))
            return False
    except Exception as e:
        print(colored("❌ Server is not running", "red"))
        print(f"   Error: {e}")
        return False

# Start the server
start_server()

🚀 Starting MCP server...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Nov/2025 18:36:31] "GET /health HTTP/1.1" 200 -


✅ MCP Server is running!
📍 Local URL: http://127.0.0.1:5000

🌐 Setting up public tunnel with ngrok...
✅ Ngrok authenticated
✅ Public URL: NgrokTunnel: "https://acronically-unharmonic-telma.ngrok-free.dev" -> "http://localhost:5000"
📍 MCP Endpoint: NgrokTunnel: "https://acronically-unharmonic-telma.ngrok-free.dev" -> "http://localhost:5000"/mcp
📍 Health Check: NgrokTunnel: "https://acronically-unharmonic-telma.ngrok-free.dev" -> "http://localhost:5000"/health

🔍 MCP Inspector Instructions:
1. Run in terminal: npx @modelcontextprotocol/inspector
2. This will open MCP Inspector in your browser
3. Enter MCP URL: NgrokTunnel: "https://acronically-unharmonic-telma.ngrok-free.dev" -> "http://localhost:5000"/mcp
4. Click 'Connect' and test the customer management tools!


t=2025-11-27T19:02:41-0600 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=4f8346a26c8f clientid=69de709f835cff9cff76869995d7c826
t=2025-11-27T19:02:41-0600 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f7d9117fd9ff err="session closed"
t=2025-11-27T20:51:21-0600 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f7d9117fd9ff err="read tcp 172.20.20.20:63365->3.12.62.205:443: read: operation timed out"
t=2025-11-27T20:51:21-0600 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-11-27T20:51:22-0600 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-11-27T2

📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response


127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 17:40:48] "POST /mcp HTTP/1.1" 200 -


📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response


t=2025-12-01T17:44:15-0600 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=515806492038 clientid=69de709f835cff9cff76869995d7c826
t=2025-12-01T17:44:15-0600 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f7d9117fd9ff err="session closed"
t=2025-12-01T17:46:11-0600 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=8edbd4f8481b clientid=69de709f835cff9cff76869995d7c826
t=2025-12-01T17:46:11-0600 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f7d9117fd9ff err="session closed"
t=2025-12-01T17:56:37-0600 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=f565645da9d5 clientid=69de709f835cff9cff76869995d7c826
t=2025-12-01T17:56:37-0600 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f7d9117fd9ff err="session closed"
t=2025-12-01T21:41:03-0600 lvl=eror msg="heart

📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP message: tools/call
📤 Sending MCP response
📥 Received MCP